In [ ]:
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

measurement = np.load('../../dataset/meas_symm_1.npz', allow_pickle=False)
n_comp = 4
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:8000]
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)

def getRealImaginaryParts(cirs):
    real = cirs[:, :, 0]
    imag = cirs[:, :, 1]
    return real, imag


def apply_pca(data, n_components):
    # data: (samples, 251, 2)
    reshaped_data = data.reshape(data.shape[0], -1)  # Now shape is (samples, 251*2) -> (12800, 502)
    
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(reshaped_data)  # (samples, 251*2) -> (12800, 502)
    
    pca = PCA(n_components=n_components)
    data_pca = pca.fit_transform(data_scaled)  # (samples, n_components)
    
    return data_pca, scaler, pca


# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)

# Extract data for ALICE and BOB channels
alice_train_CIRs = trainCIR[:, alice_channel, :, :]  
eve_train_CIRs = trainCIR[:, eve_channel, :, :] # (100, 251, 2)
print(f"alice_train_CIRs: {alice_train_CIRs.shape}")
train_cirs = np.vstack((alice_train_CIRs, eve_train_CIRs))  # (12800, 251, 2)
print(f"train_cirs: {train_cirs.shape}")
train_cirs_pca, scaler, pca = apply_pca(train_cirs, n_components=n_comp)  # (12800, n_components)
print(f"train_cirs_pca: {train_cirs_pca.shape}")


# Create labels for Alice and Eve for training
alice_train_labels = np.zeros(alice_train_CIRs.shape[0])  # Label '0' for Alice.
eve_train_labels = np.ones(eve_train_CIRs.shape[0])       # Label '1' for Eve.


# Combine data and labels for training
train_atoms = train_cirs_pca
train_labels = np.hstack((alice_train_labels, eve_train_labels))


D = train_atoms.T
print(f'Dictionary : {D.shape}')

alice_train_CIRs: (6400, 251, 2)
train_cirs: (12800, 251, 2)
train_cirs_pca: (12800, 4)
Dictionary : (4, 12800)


In [4]:
# Step 3: Extract Features for Test Data
alice_test_CIRs = testCIR[:, alice_channel, :, :]
eve_test_CIRs = testCIR[:, eve_channel, :, :]
test_cirs = np.vstack((alice_test_CIRs, eve_test_CIRs))  # (3200, 251, 2)

# Apply PCA using the same scaler and PCA fitted on training data
reshaped_test_cirs = test_cirs.reshape(test_cirs.shape[0], -1)  # (3200, 251*2)
test_cirs_scaled = scaler.transform(reshaped_test_cirs)
test_cirs_pca = pca.transform(test_cirs_scaled)  # (3200, n_components)


# Create labels for Alice and Eve for testing
alice_test_labels = np.zeros(alice_test_CIRs.shape[0])  # Label '0' for Alice.
eve_test_labels = np.ones(eve_test_CIRs.shape[0])       # Label '1' for Eve.

# Combine data and labels for testing
test_atoms = test_cirs_pca
test_labels = np.hstack((alice_test_labels, eve_test_labels))

In [5]:
# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=15):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, coefficients, class_indices, D):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep onltSample coefficients for the specified class
    reconstructed_signal = D @ coef_class
    residual = np.linalg.norm(tSample - reconstructed_signal)
    return residual

# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    coefficients = find_sparse_coefficients(tSample, D)
    
    residuals = []

    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, coefficients, class_indices, D)
        residuals.append(residual)

    # Predict the class with the smallest residual
    # Find the index of the smallest residual
    min_residual_index = np.argmin(residuals)
    # Use this index to find the corresponding class
    predicted_class = unique_classes[min_residual_index]
    
    # print(f'predicted_class : {predicted_class}')
    # print(f'residuals : {residuals}')
    return predicted_class

In [6]:
# Step 8: Classifying Test Data and Evaluating the Model
predictions = []

for testSample in test_atoms:
    predicted_class = classify_signal(testSample, D, train_labels)
    predictions.append(predicted_class)

predictions = np.array(predictions)

accuracy = np.mean(predictions == test_labels)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
print(f"\nTotal testing channel: {test_labels.shape}")

tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# # Missed Detection Rate (MDR)
MDR = fp / (fp + tn)

# # False Alarm Rate (FAR)
FAR = fn / (fn + tp)

# # Gamma calculation
gamma = (tp + fn) / (tn + fp)

# # Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp))

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")

/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear depend

Classification Accuracy: 51.50%

Total testing channel: (3200,)
tp: 1313
tn: 335
fp: 1265
fn: 287
MDR: 0.790625
FAR: 0.179375
AR: 0.515


/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear depend